In [1]:
import pandas as pd
import numpy as np

In [7]:
df1 = pd.read_parquet('COUNTY_2012.parquet')
df1 = reduce_mem_usage(df1)

Memory usage of properties dataframe is : 7433.336514472961  MB
******************************
Column:  State Code
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  County Code
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Model Year
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Min HP
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Max HP
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Min kW
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Max kW
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:

Memory usage of properties dataframe is : 7433.336514472961  MB
******************************
Column:  State Code
dtype before:  uint8
dtype after:  uint8
******************************
******************************
Column:  County Code
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Model Year
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Min HP
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Max HP
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Min kW
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Vehicle Max kW
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:

In [4]:
df2 = pd.read_parquet("COUNTY_2016.parquet")

ArrowMemoryError: realloc of size 536870912 failed

In [5]:
df1 = pd.concat((df1,df2),axis=0)

         Vehicle Fuel Type  Vehicle Count uniqueid  Year
0             Unleaded Gas              1    01000  2012
1                     None              1    01001  2012
3                     None              1    01001  2012
4                     None              1    01001  2012
5                     None              1    01001  2012
...                    ...            ...      ...   ...
19326504      Unleaded Gas              2    72153  2020
19326506      Unleaded Gas              1    72153  2020
19326507      Unleaded Gas              1    72153  2020
19326508          Electric              1    72153  2020
19326509      Unleaded Gas              1    72153  2020

[178459020 rows x 4 columns]


In [6]:
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                continue
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df

In [7]:
df1 = reduce_mem_usage(df1)

Memory usage of properties dataframe is : 4765.36994934082  MB
******************************
Column:  Vehicle Count
dtype before:  uint16
dtype after:  uint16
******************************
******************************
Column:  Year
dtype before:  uint16
dtype after:  uint16
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  4765.36994934082  MB
This is  100.0 % of the initial size


In [8]:
df1.to_parquet("Revised county file.parquet")